In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-24 22:50:48.766659: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-24 22:50:48.793911: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-24 22:50:48.960017: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-24 22:50:48.960126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-24 22:50:48.997471: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [80]:
n_samples = 32
risk_seeking = 0.1
lr = 0.01
sampler = EquationSampler(BASE_TOKENS, 16, 2)

In [81]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

In [82]:
#

In [83]:
(sequences, lengths, entropies, log_probs) = sampler.sample_sequence(n_samples)

In [84]:
ensemble = ExpressionEnsemble(sequences, lengths)

In [85]:
ensemble.optimize_constants(X, y)

15655 - DEBUG - 24-Dec-23 23:06:23 - Fitting 1/32: exp(const / const - var_x * var_x / const / const), object=ExpressionEnsemble
15655 - DEBUG - 24-Dec-23 23:06:23 - Fitting 2/32: cos(log(const - var_x / const + const) - var_x), object=ExpressionEnsemble
15655 - DEBUG - 24-Dec-23 23:06:23 - Fitting 3/32: const * sin(exp(const - const)) - const - var_x, object=ExpressionEnsemble
15655 - DEBUG - 24-Dec-23 23:06:24 - Fitting 4/32: const / var_x - var_x + const / var_x - const, object=ExpressionEnsemble
15655 - DEBUG - 24-Dec-23 23:06:24 - Fitting 5/32: log(log(var_x - const) / const + const * const), object=ExpressionEnsemble
15655 - DEBUG - 24-Dec-23 23:06:24 - Fitting 6/32: cos(const / log(const) + const / var_x * var_x), object=ExpressionEnsemble
15655 - DEBUG - 24-Dec-23 23:06:24 - Fitting 7/32: sin(var_x), object=ExpressionEnsemble
15655 - DEBUG - 24-Dec-23 23:06:24 - Fitting 8/32: sin(sin(const) - const - const - var_x) - const, object=ExpressionEnsemble
15655 - DEBUG - 24-Dec-23 23

In [86]:
rewards = ensemble.score(X, y)

In [87]:
best_epoch_expression = ensemble.expressions[tf.argmax(rewards)]
best_rewards = tf.reduce_max(rewards)

In [88]:
threshold = np.quantile(rewards, 1 - risk_seeking)
mask = rewards > threshold

In [89]:
best_rewards = rewards[mask]
best_entropies = entropies[mask]
best_log_probs = log_probs[mask]

In [95]:
risk_seeking_grad = tf.clip_by_value(
    tf.reduce_sum((best_rewards - threshold) * best_log_probs) / best_rewards.shape[0],
    -1e6,
    1e6,
)
entropy_grad = tf.clip_by_value(
    tf.reduce_sum(best_entropies) / best_rewards.shape[0], -1e6, 1e6
)

In [97]:
loss = -lr * (risk_seeking_grad + entropy_grad)